In [2]:
import numpy as np
from numpy import random
import pandas as pd
import scipy.stats as st
from pathlib import Path
import sys
sys.path.append('sobol_seq-0.1-np19py34_0/lib/python3.4/site-packages')
sys.path.append('getInvalid.py')
from sobol_seq import sobol_seq
import random
import getInvalid

**In the following cell, we figure out what the materials are called. For SW to know the material properties, it has to look them up from a specified file location. The formatting of this is difficult to get right, so we save a design study template file from solidworks, then load it here and examine the material names**

In [2]:
template=pd.read_csv("Walker_Generation_Template.csv", encoding="ISO-8859-1")
df=template.copy()
# print(df)
df.reset_index(inplace=True)  
if "index" in df.columns:
    df.drop(["index"], axis=1, inplace=True)
# print(df)
df.drop(df.index[1], inplace=True)
df.drop(df.columns[0:2], axis=1, inplace=True)

df.columns = df.iloc[0]
# print(df.columns)
df.drop(df.index[0], inplace=True)

#Define specific materials to insert into design templates
steel=" AISI 4130 Steel#^# annealed at 865C@SOLIDWORKS Materials$SOLIDWORKS Materials|AISI 4130 Steel#^# annealed at 865C|94$C:/Program Files/SOLIDWORKS Corp/SOLIDWORKS/lang/english/sldmaterials/SOLIDWORKS Materials.sldmat|AISI 4130 Steel#^# annealed at 865C|94"
aluminum=" 6061-T6 (SS)@SOLIDWORKS Materials$SOLIDWORKS Materials|6061-T6 (SS)|164$C:/Program Files/SOLIDWORKS Corp/SOLIDWORKS/lang/english/sldmaterials/SOLIDWORKS Materials.sldmat|6061-T6 (SS)|164"
titanium=" Ti-6Al-4VSolution treated and aged (SS)@SOLIDWORKS Materials$SOLIDWORKS Materials|Ti-6Al-4VSolution treated and aged (SS)|205$C:/Program Files/SOLIDWORKS Corp/SOLIDWORKS/lang/english/sldmaterials/SOLIDWORKS Materials.sldmat|Ti-6Al-4VSolution treated and aged (SS)|205"
print(steel)
print(aluminum)
print(titanium)
df.drop(df.index, axis=0, inplace=True)
# df.columns


 AISI 4130 Steel#^# annealed at 865C@SOLIDWORKS Materials$SOLIDWORKS Materials|AISI 4130 Steel#^# annealed at 865C|94$C:/Program Files/SOLIDWORKS Corp/SOLIDWORKS/lang/english/sldmaterials/SOLIDWORKS Materials.sldmat|AISI 4130 Steel#^# annealed at 865C|94
 6061-T6 (SS)@SOLIDWORKS Materials$SOLIDWORKS Materials|6061-T6 (SS)|164$C:/Program Files/SOLIDWORKS Corp/SOLIDWORKS/lang/english/sldmaterials/SOLIDWORKS Materials.sldmat|6061-T6 (SS)|164
 Ti-6Al-4VSolution treated and aged (SS)@SOLIDWORKS Materials$SOLIDWORKS Materials|Ti-6Al-4VSolution treated and aged (SS)|205$C:/Program Files/SOLIDWORKS Corp/SOLIDWORKS/lang/english/sldmaterials/SOLIDWORKS Materials.sldmat|Ti-6Al-4VSolution treated and aged (SS)|205


**We set the parameter ranges for the parameters here:**

In [3]:

scaledf=pd.DataFrame()

# scaledf["Name in Solidworks"] = [minimum,range]

scaledf["H"] = [31, 9]
scaledf["L1"] = [13, 9]
scaledf["L2"] = [5, 30]
scaledf["L3"] = [1, 10]
scaledf["L4"] = [3, 10]
scaledf["L5"] = [9, 20]
scaledf["L7"] = [7, 18]
scaledf["L8"] = [5, 10]
scaledf["A2"] = [100, 75]
scaledf["D"] = [0.05, 0.6]
scaledf["T"] = [0.05, 0.3]
scaledf["D2"] = [0.05, 0.6]
scaledf["T2"] = [0.05, 0.3]
scaledf["D3"] = [0.25, 0.7]
#material
scaledf.index=["min", "factor"]
scaledf=scaledf.transpose()

**We create the sobol sequence an scale the sobol vectors by the scale factors defined in the previous cell.**

In [4]:
seqlength=55000
sobolvec= sobol_seq.i4_sobol_generate(len(scaledf.index), seqlength)

scalefactor = scaledf["factor"].values
scalemin = scaledf["min"].values
unscaled = np.array(sobolvec)

expL3 = 1
expL4 = 1

scaled = []
for row in unscaled:
    scaled_row = []
    for i, value in enumerate(row):
        if(scaledf.index[i] == "L3"):
            scaled_value = (value**expL3) * scalefactor[i] + scalemin[i]
        elif(scaledf.index[i] == "L4"):
            scaled_value = (value**expL4) * scalefactor[i] + scalemin[i]
        else:
            scaled_value = value * scalefactor[i] + scalemin[i]
        scaled_row.append(scaled_value)
    scaled.append(scaled_row)


# scaled = unscaled * scalefactor + scalemin

scaled = pd.DataFrame(scaled, columns = scaledf.index )

# print(np.shape(res))
# numconfigs=1000
# numparams=len(res[0,:])
# configs=np.zeros((numconfigs,numparams))
# i=0
# sobidx=0
# while i<(numconfigs) and sobidx<seqlength:
#     config=res[sobidx,:]
#     sobidx+=1
#     if checkconfig(config):
#         configs[i,:]=config
#         if config[0]==float("NaN"):
#             print(res[sobidx,:])
#         i+=1
# configdf=pd.DataFrame(configs, columns=cols)
print(scaled)

               H         L1         L2        L3         L4         L5  \
0      35.500000  17.500000  20.000000  6.000000   8.000000  19.000000   
1      37.750000  15.250000  27.500000  3.500000  10.500000  14.000000   
2      33.250000  19.750000  12.500000  8.500000   5.500000  24.000000   
3      34.375000  16.375000  23.750000  2.250000  11.750000  26.500000   
4      38.875000  20.875000   8.750000  7.250000   6.750000  16.500000   
...          ...        ...        ...       ...        ...        ...   
54995  35.420486  15.264694  27.787018  9.625641   8.222015  18.964905   
54996  39.920486  19.764694  12.787018  4.625641   3.222015  28.964905   
54997  37.670486  13.014694   5.287018  7.125641   5.722015  13.964905   
54998  33.170486  17.514694  20.287018  2.125641  10.722015  23.964905   
54999  32.607986  16.952194  10.912018  4.000641  12.597015  22.714905   

              L7         L8          A2         D         T        D2  \
0      16.000000  10.000000  137.50000

**We randomize a few parameters that are not continuous variables. For example, we have three discrete materials. We also have two boolean parameters (true/false) governing whether we have seat stay and chain stay bridges on the bike**

In [5]:
# count=0
walkers=[]
df=scaled.copy()
df.columns = [" " + string for string in list(df.columns)]
        
for idx in df.index:
    matnum = random.randint(0,2)
    if matnum==0:
        df.at[idx, " CrossMat"] = steel
    if matnum==1:
        df.at[idx, " CrossMat"] = aluminum
    if matnum==2:
        df.at[idx, " CrossMat"] = titanium
    matnum = random.randint(0,2)
    if matnum==0:
        df.at[idx, " FrameMat"] = steel
    if matnum==1:
        df.at[idx, " FrameMat"] = aluminum
    if matnum==2:
        df.at[idx, " FrameMat"] = titanium
    # if (df.at[idx, " D2"] + df.at[idx, " T2"]) > (df.at[idx, " D"] + df.at[idx, " T"]): 
    #     df.at[idx, " D2"] = df.at[idx, " D"]
    #     df.at[idx, " T2"] = df.at[idx, " T"]
        
# print(df)


invalid_walkers, perc=getInvalid.getInvalid(df)
print(len(invalid_walkers))
print(invalid_walkers)
df.drop(invalid_walkers, inplace=True)

# print(np.array(bridges))
# np.save("./Batch Info/bridges", np.array(bridges))

36476
[1, 5, 6, 7, 10, 11, 12, 13, 14, 15, 16, 17, 19, 20, 21, 26, 28, 30, 31, 32, 34, 37, 38, 39, 40, 42, 43, 44, 45, 46, 47, 49, 50, 51, 52, 53, 57, 58, 59, 60, 62, 63, 64, 66, 68, 69, 71, 74, 80, 82, 84, 85, 86, 87, 91, 92, 93, 95, 96, 98, 100, 101, 103, 104, 106, 107, 108, 109, 110, 112, 114, 116, 117, 118, 119, 120, 121, 122, 125, 127, 128, 129, 130, 134, 135, 136, 137, 139, 141, 143, 145, 146, 147, 148, 150, 152, 153, 154, 155, 158, 160, 163, 165, 166, 167, 168, 169, 171, 172, 173, 174, 175, 178, 179, 180, 181, 182, 183, 184, 185, 187, 188, 190, 191, 193, 194, 195, 197, 198, 199, 200, 201, 202, 203, 204, 206, 207, 208, 209, 211, 214, 216, 217, 218, 221, 222, 223, 225, 227, 228, 229, 230, 232, 238, 239, 240, 241, 242, 243, 246, 248, 250, 254, 255, 256, 257, 258, 260, 261, 262, 263, 269, 271, 272, 274, 275, 276, 277, 280, 281, 282, 283, 284, 285, 286, 287, 288, 290, 292, 294, 295, 301, 303, 306, 308, 309, 310, 313, 314, 317, 318, 319, 320, 321, 322, 323, 325, 328, 330, 332, 333, 33

**Save the file for reference, then load it back up (avoids having to run the above code every time we generate a batch of designs to simulate). We typically simulate ~200 at a time otherwise SW gets overwhelmed (max 500).**

In [6]:
df.to_csv("FINALProcessed_structural_data_sobol.csv")

In [4]:
import numpy as np
from numpy import random
import pandas as pd
import scipy.stats as st
from pathlib import Path
import random
import getInvalid

augdf=pd.read_csv("tistdf.csv", index_col=0)

augdf=augdf.iloc[400:600]
print(augdf)

               H         L1         L2         L3         L4         L5  \
10257  38.601990  14.549622  12.622681   7.067505  12.955444  14.279541   
10297  39.023865  14.971497  21.528931   8.473755  11.549194  25.842041   
10397  35.894958  15.569153  19.067993  10.778442  12.760132  28.420166   
10399  31.535583  13.178528   6.411743   9.372192  10.103882  20.607666   
10444  38.496521  18.311340  22.114868   7.106567  10.807007  23.888916   
...          ...        ...        ...        ...        ...        ...   
15885  36.080627  15.119812  22.836304   5.044800   3.830688  19.423584   
15906  33.690002  20.041687  18.617554   9.888550   6.174438  17.861084   
15957  37.697815  17.018250  16.508179   5.435425  12.815063  13.017334   
16102  32.459534  18.459656  18.031616  10.865112  12.385376  11.845459   
16118  32.740784  20.990906  26.469116   6.802612  10.197876  18.720459   

              L7         L8          A2         D         T        D2  \
10257   7.862427  14.73327

**Using the "Design Study Template" File, we cram the generated designs into the correct format for SW to read, then save**

In [5]:
# for idx in augdf.index:
#     if augdf.at[idx, " CS F"]==augdf.at[idx, " BB Length"]/2:
#         augdf.at[idx, " CS F"]=augdf.at[idx, " CS F"]-.0001
#         print("Adjusting CS F for model " + str(idx) + " by 0.0001 since it is half of BB length and would cause model to lock!")


template=pd.read_csv("Walker_Generation_Template.csv", encoding="ISO-8859-1")
# if "index" in template.columns:
#     template.drop(["index"], axis=1, inplace=True)
newindex=augdf.columns.insert(0,"Status")
augdf["Status"]=np.full(len(augdf.index), 1)
augdf=augdf[newindex]
augdf.index=["Walker" + str(i) for i in augdf.index]
# print(template)
template.reset_index(inplace=True)  
units=template.iloc[1]

#Old code to drop "Level_0", replace this by dropping entries from top of units until reaching correct length
#     units.drop(["level_0"], axis=0, inplace=True)

while len(units.index)>len(augdf.columns):
    units.drop([units.index[0]], axis=0, inplace=True)
units.index=augdf.columns
units.rename("Units", inplace=True)
augdf=augdf.append(units)
newindices= augdf.index[:-1].insert(0, augdf.index[-1])
augdf=augdf.reindex(newindices)
augdf.columns.rename("Parameters", inplace=True)

augdf=augdf.T.reset_index().T
cols=[len(augdf.index)-2, len(augdf.columns)-1] + [""]*(len(augdf.columns)-2)
# print(cols)
augdf.columns=cols
augdf.columns.rename("Design Set", inplace=True)
# print(units)
print(augdf)

Design Set      200         16                                              \
Parameters   Status          H         L1         L2         L3         L4   
Units                       in         in         in         in         in   
Walker10257       1   38.60199  14.549622  12.622681   7.067505  12.955444   
Walker10297       1  39.023865  14.971497  21.528931   8.473755  11.549194   
Walker10397       1  35.894958  15.569153  19.067993  10.778442  12.760132   
...             ...        ...        ...        ...        ...        ...   
Walker15885       1  36.080627  15.119812  22.836304     5.0448   3.830688   
Walker15906       1  33.690002  20.041687  18.617554    9.88855   6.174438   
Walker15957       1  37.697815   17.01825  16.508179   5.435425  12.815063   
Walker16102       1  32.459534  18.459656  18.031616  10.865112  12.385376   
Walker16118       1  32.740784  20.990906  26.469116   6.802612  10.197876   

Design Set                                                     

C:\Users\advai\AppData\Local\Temp\ipykernel_100324\723245278.py:22: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  units.drop([units.index[0]], axis=0, inplace=True)
C:\Users\advai\AppData\Local\Temp\ipykernel_100324\723245278.py:25: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  augdf=augdf.append(units)


In [6]:
augdf.to_csv("generated_param_study_template.csv", index_label="Design Set")